In [ ]:
class To_Uniform(nn.Module):
    def __init__(self,
                 encoder_layers=[64, 256, 128, 32, 16, 8, 3],
                 decoder_layers=[3, 8, 16, 32, 128, 256, 64],
                 encoder_act=nn.ReLU,
                 decoder_act=nn.ReLU,
                 final_encoder_act=nn.Sigmoid,
                 final_decoder_act=nn.Sigmoid,
                 use_batchnorm=True):
        super().__init__()

        self.encoder = self.build_mlp(encoder_layers, encoder_act, final_encoder_act, use_batchnorm)
        self.decoder = self.build_mlp(decoder_layers, decoder_act, final_decoder_act, use_batchnorm=False)

    def build_mlp(self, layer_sizes, activation_fn, final_activation, use_batchnorm=False):
        layers = []
        for i in range(len(layer_sizes) - 2):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            layers.append(activation_fn())
            if use_batchnorm:
                layers.append(nn.BatchNorm1d(layer_sizes[i+1]))

        # Final Layer
        layers.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))
        if final_activation is not None:
            layers.append(final_activation())
        return nn.Sequential(*layers)

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return z, x_hat

    def criterion(self, z_pred, z_true, x_pred, x_true):
        return - 1.0 * nn.MSELoss()(z_pred, z_true) + 1.0 * nn.MSELoss()(x_pred, x_true)